## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology and Analysis](#methodologyAndAnalysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

When an foreign investor would like to invest in their local culture cuisine in another country, in this case Toronto, could be very challeging without any local guildance. In order for a restuarant to be successfuly, delicious food is not the only ingrdediance. The location of the restaurant plays a hugh role since it dictates the type of customers, for exmaple commerical area usually would brings in working class during during lunch time and school area would brings in students. In addition, the cost of commerical leasing is extremely pricy in Toronto Area. Obveriously, an investor would need to pick a location that is most cost effective.

In our business case, an Indonesian investor wishes to open a take-out/catering restaurant business in Toronto. The menu would feature authentic Indonesian cruises which cater towards not only customers of Indonesian origin but also foodie who are adventurous about food with complex flavours. The ideal location would be close to major Highway, Indonesian community and no other Indonesian similar restaurant to compete with. The goal of this project is to find out which is the best location of this restaurant that fits above requirements. Our major audience would be an foreign investor who would wish to open a restaurant in Toronto.

## Data <a name="data"></a>

Based on Definition of our problem, factos that will influence our decission are:

* number of indnesian community around an indonesian grocery store Oey Trading Co.
* number of indnesian or southeat asian restaurant around the neighoord
* distance from major highway

The first step is to use wikipedia data to obtain Toronto Postal code and neighborhood. There is a hotspot known for indnesian which is a Indonesian grocery store Oey Trading Co. located 3241 Kennedy Rd#12 which we will explore. Based on the nearby Indonesian community, we locate the potential neighborhood that fits our business project.

Once we narrow down the possible locations, we will pick a location that

* has the least number of southeast asia restaurant competition and closest to the indonesian communities using Foursquare API
* the closest distance from the highway

In addition, we will use historical commercial leasing data from Toronto Regional Real Estate Board report to determine the most cost efficient location to open up the business.

In [1]:
# Scrap Data from Wikipedia
import pandas as pd
import requests

!conda install -c anaconda beautifulsoup4 --yes

from bs4 import BeautifulSoup

# Use beautifulsoup to scrap wiki table
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
    
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
# Remove Not assigned in Borough
indexNames = df[df['Borough'] == 'Not assigned'].index
df.drop(indexNames , inplace=True)

# Replace Not assigned in Neighbourhood with Borough value
import numpy as np
df["Neighbourhood"] = np.where(df["Neighbourhood"] == "Not assigned", df["Borough"],df["Neighbourhood"])

df.shape
df.head(20)

# Import coordinates csv file into data frame

!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

df_geo = pd.read_csv("Geospatial_Coordinates.csv")

df_final = pd.merge(df, df_geo, on = "Postal Code")


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    beautifulsoup4-4.9.3       |     pyhb0f4dca_0          87 KB  anaconda
    ca-certificates-2020.10.14 |                0         128 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    soupsieve-2.0.1            |             py_0          33 KB  anaconda
    ------------------------------------------------------------
                                           Total:         407 KB

The following NEW packages will be INSTALLED:

  beautifulsoup4     anaconda/noarch::beautifulsoup4-4.9.3-pyhb0f4dca_0
  soupsieve          anaconda/noarch::soupsieve-2.0.1-py_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates 

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.0
  latest version: 4.10.1

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.12.5  |       ha878542_0         137 KB  conda-forge
    certifi-2020.12.5          |   py36h5fab9bb_1         143 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.1.0                |     pyhd3deb0d_0          64 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         378 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geograph

In [3]:
df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [4]:
# Use geopy library to get the latitude and longitude values of Toronto
address = 'Toronto, TO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of Toronto using latitude and longitude values
map_to = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_final['Latitude'], df_final['Longitude'], df_final['Borough'], df_final['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_to)  
    
map_to


The geograpical coordinate of Toronto are 43.65238435, -79.38356765.


In [5]:
# Define foursquare credentails

CLIENT_ID = 'BX2KDMWEEZOI0JVJAWR3MVSYV524N22JY0VQEH5EM0ID4QVR' # your Foursquare ID
CLIENT_SECRET = '0R0UNLVF50ACZ2OTTDL3QJBGBJISN0YC5VRX0QDBTZWXBCYR' # your Foursquare Secret
ACCESS_TOKEN = 'Y5Y2PZBELCROL51VZQ4SGEDQBGQJ2P05GTG2FVV5LRKJGWME' # your Foursquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: ' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BX2KDMWEEZOI0JVJAWR3MVSYV524N22JY0VQEH5EM0ID4QVR
CLIENT_SECRET: 0R0UNLVF50ACZ2OTTDL3QJBGBJISN0YC5VRX0QDBTZWXBCYR


In [6]:
# Lets explore around an Indonesian supermarket Oey trading co. addres:3241 Kennedy Road #12, Scarborough, ON M1V 2J8

address = '3241 Kennedy Road, Scarborough, ON, TO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.81457145863235 -79.30284890361509


In [7]:
search_query = 'indonesia'
radius = 10000
print(search_query + ' .... OK!')


indonesia .... OK!


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=BX2KDMWEEZOI0JVJAWR3MVSYV524N22JY0VQEH5EM0ID4QVR&client_secret=0R0UNLVF50ACZ2OTTDL3QJBGBJISN0YC5VRX0QDBTZWXBCYR&ll=43.81457145863235,-79.30284890361509&oauth_token=Y5Y2PZBELCROL51VZQ4SGEDQBGQJ2P05GTG2FVV5LRKJGWME&v=20180605&query=indonesia&radius=10000&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607b35f3fc6a4d727bd72d17'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4d45a4c1befe236a552801e3',
    'name': 'Toko Indonesia',
    'location': {'lat': 43.81661479154673,
     'lng': -79.30284102884998,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.81661479154673,
       'lng': -79.30284102884998}],
     'distance': 227,
     'cc': 'CA',
     'country': 'Canada'},
    'categories': [{'id': '4bf58dd8d48988d118951735',
      'name': 'Grocery Store',
      'pluralName': 'Grocery Stores',
      'shortName': 'Grocery Store',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1618687475',
    'hasPerk': False},
   {'id': '544ab0f2498e767f774145df',
    'name': 'Taste of Indonesia',
    'location': {'lat': 43.816364,
     'lng': -79.303526,
     'labeledLatLngs': [{'l

In [10]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.country,location.address,location.postalCode,location.city,location.state,location.formattedAddress
0,4d45a4c1befe236a552801e3,Toko Indonesia,"[{'id': '4bf58dd8d48988d118951735', 'name': 'G...",v-1618687475,False,43.816615,-79.302841,"[{'label': 'display', 'lat': 43.81661479154673...",227,CA,Canada,NaN,NaN,NaN,NaN,NaN
1,544ab0f2498e767f774145df,Taste of Indonesia,[],v-1618687475,False,43.816364,-79.303526,"[{'label': 'display', 'lat': 43.816364, 'lng':...",206,CA,Canada,NaN,NaN,NaN,NaN,NaN
2,55aa7ef8498e5e476ceab7fe,Konjen Indonesia Toronto,[],v-1618687475,False,43.759667,-79.381651,"[{'label': 'display', 'lat': 43.759667, 'lng':...",8801,CA,Canada,NaN,NaN,NaN,NaN,NaN
3,531e538a498eb81d6bdb74fe,"Wisma Konsulat Jendral Republik Indonesia, Tor...",[],v-1618687475,False,43.759845,-79.381588,"[{'label': 'display', 'lat': 43.759845, 'lng':...",8783,CA,Canada,NaN,NaN,NaN,NaN,NaN
4,52041bd6498e140423b621bb,Kediaman Konsulat Jendral Indonesia - Toronto,[],v-1618687475,False,43.758490,-79.384941,"[{'label': 'display', 'lat': 43.75849, 'lng': ...",9082,CA,Canada,NaN,NaN,NaN,NaN,NaN


In [11]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,lat,lng,labeledLatLngs,distance,cc,country,address,postalCode,city,state,formattedAddress,id
0,Toko Indonesia,Grocery Store,43.816615,-79.302841,"[{'label': 'display', 'lat': 43.81661479154673...",227,CA,Canada,NaN,NaN,NaN,NaN,NaN,4d45a4c1befe236a552801e3
1,Taste of Indonesia,None,43.816364,-79.303526,"[{'label': 'display', 'lat': 43.816364, 'lng':...",206,CA,Canada,NaN,NaN,NaN,NaN,NaN,544ab0f2498e767f774145df
2,Konjen Indonesia Toronto,None,43.759667,-79.381651,"[{'label': 'display', 'lat': 43.759667, 'lng':...",8801,CA,Canada,NaN,NaN,NaN,NaN,NaN,55aa7ef8498e5e476ceab7fe
3,"Wisma Konsulat Jendral Republik Indonesia, Tor...",None,43.759845,-79.381588,"[{'label': 'display', 'lat': 43.759845, 'lng':...",8783,CA,Canada,NaN,NaN,NaN,NaN,NaN,531e538a498eb81d6bdb74fe
4,Kediaman Konsulat Jendral Indonesia - Toronto,None,43.758490,-79.384941,"[{'label': 'display', 'lat': 43.75849, 'lng': ...",9082,CA,Canada,NaN,NaN,NaN,NaN,NaN,52041bd6498e140423b621bb
5,Indonesian Reformed Evangelical Church (IREC) ...,None,43.863461,-79.383443,"[{'label': 'display', 'lat': 43.86346054517607...",8455,CA,Canada,NaN,NaN,NaN,NaN,NaN,567db771498e89ea711bcb72
6,Indonesian Reformed Evangelical Church Trinity,Church,43.760860,-79.386060,"[{'label': 'display', 'lat': 43.76086, 'lng': ...",8970,CA,Canada,2737 Bayview Avenue,M2L 1C5,Toronto,ON,"[2737 Bayview Avenue, Toronto ON M2L 1C5]",587bdd036cda1c20bb27555a
7,Warung Kampoeng,Indonesian Restaurant,43.741985,-79.222577,"[{'label': 'display', 'lat': 43.74198464874808...",10340,CA,Canada,3143 Eglinton Ave E,NaN,Toronto,ON,"[3143 Eglinton Ave E, Toronto ON]",5262bc9311d2f51f54f035ef


In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around Oey Trading Co.

# add a red circle marker to represent the Oey Trading Co.
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Oey Trading Co.',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Southeast Asian Restaurant as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Based on the Initial seach result, there is an Indionesian Reformed Evangelicial Church located near Bayview Aveune and Highway 401 entrance. Lets explore the mid point Mid point 


In [13]:
mid_lag = (43.81457145863235+43.760860)/2
mid_long = (-79.30284890361509-79.386060)/2
mid_radius = 1000

In [14]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, mid_lag, mid_long, VERSION, mid_radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=BX2KDMWEEZOI0JVJAWR3MVSYV524N22JY0VQEH5EM0ID4QVR&client_secret=0R0UNLVF50ACZ2OTTDL3QJBGBJISN0YC5VRX0QDBTZWXBCYR&ll=43.787715729316176,-79.34445445180754&v=20180605&radius=1000&limit=100'

In [15]:
import requests
results_mid = requests.get(url).json()
'There are {} around around.'.format(len(results_mid['response']['groups'][0]['items']))

'There are 32 around around.'

In [16]:
items_mid = results_mid['response']['groups'][0]['items']
items_mid[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4e5f682a1850b75573f552dc',
  'name': "Allan's Pastry Shop",
  'location': {'address': '3056 Don Mills Rd.',
   'lat': 43.7856560376599,
   'lng': -79.35231415152528,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.7856560376599,
     'lng': -79.35231415152528}],
   'distance': 671,
   'postalCode': 'M2J 3C1',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['3056 Don Mills Rd.', 'Toronto ON M2J 3C1', 'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-4e5f682a1850b75573f552dc-0'}

In [17]:
dataframe_mid = json_normalize(items_mid) # flatten JSON

# filter columns
filtered_columns_mid = ['venue.name', 'venue.categories'] + [col for col in dataframe_mid.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered_mid = dataframe_mid.loc[:, filtered_columns_mid]

# filter the category for each row
dataframe_filtered_mid['venue.categories'] = dataframe_filtered_mid.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered_mid.columns = [col.split('.')[-1] for col in dataframe_filtered_mid.columns]

dataframe_filtered_mid.head(30)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Allan's Pastry Shop,Bakery,3056 Don Mills Rd.,43.785656,-79.352314,"[{'label': 'display', 'lat': 43.7856560376599,...",671,M2J 3C1,CA,Toronto,ON,Canada,"[3056 Don Mills Rd., Toronto ON M2J 3C1, Canada]",NaN,NaN,4e5f682a1850b75573f552dc
1,Mr Jerk,Caribbean Restaurant,3050 Don Mills Rd,43.785932,-79.352842,"[{'label': 'display', 'lat': 43.78593238518595...",702,NaN,CA,Toronto,ON,Canada,"[3050 Don Mills Rd, Toronto ON, Canada]",NaN,NaN,4c129019a5eb76b01b69beb7
2,Tim Hortons,Coffee Shop,1500 Finch Ave E,43.792905,-79.354490,"[{'label': 'display', 'lat': 43.79290521661804...",991,M2J 4Y6,CA,Toronto,ON,Canada,"[1500 Finch Ave E (at Don Mills Rd), Toronto O...",at Don Mills Rd,North York,4fa2dec2e4b07ed218b54d11
3,Subway,Sandwich Place,"3030 Don Mills Rd East, Unit 27",43.785831,-79.351953,"[{'label': 'display', 'lat': 43.78583089500218...",638,M2J 3C1,CA,Toronto,ON,Canada,"[3030 Don Mills Rd East, Unit 27 (in Peanut Pl...",in Peanut Plaza,NaN,4bc8596e14d7952144d768e9
4,The Beer Store,Beer Store,3078 Don Mills Rd,43.786126,-79.352628,"[{'label': 'display', 'lat': 43.7861261, 'lng'...",680,M2J 3C1,CA,North York,ON,Canada,"[3078 Don Mills Rd (Van Horne), North York ON ...",Van Horne,NaN,4c5cbeb22815c9286e1ab367
5,Petro-Canada,Gas Station,3075 Don Mills Rd,43.787866,-79.352850,"[{'label': 'display', 'lat': 43.78786608693070...",674,M2J 3C2,CA,Toronto,ON,Canada,"[3075 Don Mills Rd (at Van Horne Ave), Toronto...",at Van Horne Ave,NaN,4c35ac5c452620a1650c270f
6,Fairview Library Theatre,Theater,35 Fairview Mall Drive,43.779018,-79.346526,"[{'label': 'display', 'lat': 43.77901805702380...",982,NaN,CA,Toronto,ON,Canada,"[35 Fairview Mall Drive, Toronto ON, Canada]",NaN,NaN,4bbe84be82a2ef3b37912bd2
7,Rexall Pharma Plus,Pharmacy,3555 Don Mills Rd.,43.793962,-79.353240,"[{'label': 'display', 'lat': 43.79396236392167...",990,M2H 3N3,CA,Toronto,ON,Canada,"[3555 Don Mills Rd. (at Finch Ave. E.), Toront...",at Finch Ave. E.,NaN,4b7ae34ef964a520e2432fe3
8,Popeyes Louisiana Kitchen,Fried Chicken Joint,"3070 Don Mills Rd, #13a, Peanut Plaza",43.785597,-79.352291,"[{'label': 'display', 'lat': 43.78559715766426...",672,M2J 3C1,CA,North York,ON,Canada,"[3070 Don Mills Rd, #13a, Peanut Plaza (at Pea...",at Peanut Plaza,NaN,4b7af60ff964a5208b482fe3
9,Pizza Pizza,Pizza Place,3040 DON MILLS ROAD,43.785946,-79.352942,"[{'label': 'display', 'lat': 43.78594607852859...",709,M2J 3C3,CA,Don Mills,ON,Canada,"[3040 DON MILLS ROAD, Don Mills ON M2J 3C3, Ca...",NaN,NaN,4d8ba6930c4e41bdb2f9667f


Let take a look at the mid point area around the map

In [18]:
mid_venues_map = folium.Map(location=[mid_lag, mid_long], zoom_start=13) # generate map centred around mid point


# add mid point as a red circle mark
folium.CircleMarker(
    [mid_lag, mid_long],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(mid_venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered_mid.lat, dataframe_filtered_mid.lng, dataframe_filtered_mid.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(mid_venues_map)

# display map
mid_venues_map

In [19]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
venues = results_mid['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  from ipykernel import kernelapp as app


,name,categories,lat,lng
0,Allan's Pastry Shop,Bakery,43.785656,-79.352314
1,Mr Jerk,Caribbean Restaurant,43.785932,-79.352842
2,Tim Hortons,Coffee Shop,43.792905,-79.354490
3,Subway,Sandwich Place,43.785831,-79.351953
4,The Beer Store,Beer Store,43.786126,-79.352628


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

32 venues were returned by Foursquare.


In [21]:
#Explore Neighbourhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
# Select Borough that contain "M2" into dataframe to_data
to_data = df_final[df_final['Postal Code'].str.contains("M2")].reset_index(drop=True)
to_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"York Mills, Silver Hills",43.757490,-79.374714
4,M2M,North York,"Willowdale, Newtonbrook",43.789053,-79.408493


In [23]:
# new dataframe called to_venues.
to_venues = getNearbyVenues(names=to_data['Neighbourhood'],
                                   latitudes=to_data['Latitude'],
                                   longitudes=to_data['Longitude']
                                  )

print(to_venues.shape)
to_venues.head()

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
(119, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,New York Fries,43.803664,-79.363905,Fast Food Restaurant
1,Hillcrest Village,43.803762,-79.363452,Eagle's Nest Golf Club,43.805455,-79.364186,Golf Course
2,Hillcrest Village,43.803762,-79.363452,AY Jackson Pool,43.804515,-79.366138,Pool
3,Hillcrest Village,43.803762,-79.363452,Villa Madina,43.801685,-79.363938,Mediterranean Restaurant
4,Hillcrest Village,43.803762,-79.363452,Duncan Creek Park,43.805539,-79.360695,Dog Run


## Methodology and Analysis <a name="methodologyAndAnalysis"></a>

In [24]:
# Analyze Each Neighborhood
# one hot encoding
to_onehot = pd.get_dummies(to_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
to_onehot['Neighbourhood'] = to_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

to_onehot.head()

,Neighbourhood,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court,Golf Course,Greek Restaurant,Grocery Store,Hotel,Ice Cream Shop,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Martial Arts School,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [25]:
#group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

to_grouped = to_onehot.groupby('Neighbourhood').mean().reset_index()
to_grouped

,Neighbourhood,Accessories Store,American Restaurant,Arts & Crafts Store,Asian Restaurant,Bakery,Bank,Bar,Baseball Field,Boutique,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Butcher,Café,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court,Golf Course,Greek Restaurant,Grocery Store,Hotel,Ice Cream Shop,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Luggage Store,Martial Arts School,Mediterranean Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Plaza,Pool,Ramen Restaurant,Restaurant,Salon / Barbershop,Sandwich Place,Shoe Store,Shopping Mall,Spa,Sporting Goods Shop,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Fairview, Henry Farm, Oriole",0.014706,0.014706,0.000000,0.014706,0.029412,0.029412,0.014706,0.014706,0.014706,0.000000,0.014706,0.014706,0.014706,0.000000,0.000000,0.014706,0.014706,0.102941,0.088235,0.014706,0.014706,0.014706,0.014706,0.000000,0.0,0.014706,0.058824,0.014706,0.0,0.014706,0.000000,0.000000,0.000000,0.029412,0.014706,0.044118,0.000000,0.014706,0.000000,0.014706,0.0,0.0,0.000000,0.014706,0.014706,0.014706,0.0,0.000000,0.014706,0.000000,0.000000,0.0,0.000000,0.044118,0.014706,0.014706,0.029412,0.014706,0.014706,0.014706,0.000000,0.014706,0.000000,0.014706,0.014706,0.014706,0.014706,0.000000,0.029412
2,Hillcrest Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.200000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.2,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Willowdale South,0.000000,0.000000,0.030303,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.030303,0.0,0.030303,0.030303,0.000000,0.0,0.000000,0.030303,0.030303,0.030303,0.030303,0.000000,0.030303,0.030303,0.000000,0.030303,0.000000,0.0,0.0,0.030303,0.000000,0.000000,0.030303,0.0,0.030303,0.000000,0.060606,0.030303,0.0,0.090909,0.030303,0.000000,0.030303,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.030303,0.000000
4,Willowdale West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.166667,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.0000

In [26]:
# Each neighborhood along with the top 10 most common venues

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = to_grouped['Neighbourhood']

for ind in np.arange(to_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(to_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head(10)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Dog Run,Cosmetics Shop,Department Store,Dessert Shop,Discount Store
1,"Fairview, Henry Farm, Oriole",Clothing Store,Coffee Shop,Fast Food Restaurant,Juice Bar,Restaurant,Japanese Restaurant,Bakery,Bank,Shoe Store,Women's Store
2,Hillcrest Village,Pool,Golf Course,Dog Run,Fast Food Restaurant,Mediterranean Restaurant,Food Court,Greek Restaurant,Electronics Store,Chocolate Shop,Grocery Store
3,Willowdale South,Coffee Shop,Ramen Restaurant,Café,Pizza Place,Shopping Mall,Lounge,Korean Restaurant,Middle Eastern Restaurant,Juice Bar,Movie Theater
4,Willowdale West,Pharmacy,Grocery Store,Coffee Shop,Butcher,Supermarket,Pizza Place,Dessert Shop,Clothing Store,Convenience Store,Cosmetics Shop
5,York Mills West,Convenience Store,Park,Women's Store,Dog Run,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Electronics Store,Clothing Store
6,"York Mills, Silver Hills",Martial Arts School,Women's Store,Electronics Store,Convenience Store,Cosmetics Shop,Department Store,Dessert Shop,Discount Store,Dog Run,Fast Food Restaurant


## Results and Discussion <a name="results"></a>

At first, we located Indonesian communities around the neighborhood around an Indonesian grocery store Oey Trading Co. we have located one Indonesian restaurant Warung Kampoeng 10.3km south east and an Indonesian church Indonesian Reformed Evangelical Church (IREC) 8.8km south west. These are the only search results from foursquare, which indicated Indonesian communities are very small in the neighborhood.
The midpoint between Oey Trading Co. and IREC was explored, since our goal is to have the least amount of competition and being near a church community would definitely benefit the restaurant. The neighborhood for the midpoint is identified as “Toronto Central” which has postal code starting “M2*”. 
The Toronto Central search returned 8 neighborhoods which are Hillcrest Village, Fairview, Henry Farm, Oriole, Bayview village, York mills, Silver Hills, Willowdale, Newtonbrook, Willowdale South, York Mills, West and Willowdale West. 
The top 10 common venues for each Neighborhoods were listed, the area that has the most restaurants are Bayview Village, Fairview/Henry Farm/Oriole, Willowdale South. However, only Willowdale South has Vietnamese Restaurant which is the only southeast Asia cuisine competition and it is the 8th most common venue in the area. While Willowdale West and York Mills have the least counts of restaurants, they are the furthest from the Indonesian supermarket. The only location that fits all the requirements would be Fairview/Henry Farm//Oriole which is not only close to both major highways 401 and 404 and also close to the subway and a Major Mall in Toronto. 
According to Toronto Regional Real Estate Board February 2021 report, Retail Leasing in Fairview/Henry Farm/Oriole was 20 CAD/SF, whereas Willowdale and York Mills were 34.29 CAD/SF Net and 46.50 CAD/SF respectively.

This Project studied the best location to open up an Indonesian restaurant in Toronto. The data broadly narrowed down the Toronto Central area as the ideal location. Further investigation into the neighborhoods, parameters such as  distances from Highway, local venues such as Mall, subway, other southeast Asia restaurants and rent cost suggested that Fairview/Henry Farm/Oriole is the ideal location to open up and Take out/cater Indonesian restaurant business.

## Conclusion <a name="conclusion"></a>

In conclusion, the most ideal neighborhood in Toronto to open up an Indonesian take out/cater restaurant is Fairview/Henry Farm/Oriole. The data very have looked at is very limited, since Indonesian community is rather small in Toronto. There isn’t enough data to justify using statistical analyst or Machine Learning to generate a predictive model. 